### Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import fuzzywuzzy

### Abrindo os dados de ruas da CET

In [ ]:
ruas_cet = pd.DataFrame.spatial.from_featureclass("ruas_cet_sp")

In [ ]:
def remove_espacos_em_branco(string):
    return ' '.join([x for x in string.split(' ') if len(x)>0])
pd.DataFrame(ruas_cet['cvc_nomelg'].apply(lambda x: remove_espacos_em_branco(x) ).unique()).sort_values(0).to_csv(r"D:\Trabalho\faculdade\TCC\outputs\preprocessamento\georreferenciamento\classeviaria_cet.csv", index=False)

### Abrindo os dados de lentidão da CET

In [ ]:
lentidao_cet = pd.read_csv(r"D:\Trabalho\faculdade\TCC\dados\lentidao\baixados\lentidaotrechos2019.csv",sep=';',encoding='latin-1')

Criando DataFrame com os nomes (únicos) dos corredores

In [ ]:
corredores_lentidao_cet = pd.DataFrame(lentidao_cet["corredor"].unique()).rename(columns={0:'corredor'})
corredores_lentidao_cet

## Fazendo Fuzzy Join

### Lentidão com ruas da CET

Criando função que faz o Fuzzy Join

In [ ]:
from fuzzywuzzy import process

def fuzzy_merge(df_1, df_2, key1, key2, threshold=60, limit=3):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1,m

Fazendo join dos corredores (lentidão CET) com as vias do Dataset de ruas da CET

#### Considerando coluna "cvc_nmlogr"

In [ ]:
#Análise feita considerando apenas o nome "puro" da rua. Ou seja, sem prefixos como Av, R. Etc.
fuzzy_merge,limiares = fuzzy_merge(corredores_lentidao_cet,ruas_cet,"corredor","cvc_nmlogr")

In [ ]:
fuzzy_merge.head(3)

In [ ]:
#Criando colunas que contém os nomes de ruas que deram match
match1_lista, match2_lista, match3_lista = [],[],[]
for matches_trio in fuzzy_merge['matches']:
    match1,match2,match3 = matches_trio.split(', ')[0],matches_trio.split(', ')[1],matches_trio.split(', ')[2]
    match1_lista.append(match1)
    match2_lista.append(match2)
    match3_lista.append(match3)

#Criando colunas que contém os graus de similaridade de cada rua
limiar1_lista,limiar2_lista,limiar3_lista = [],[],[]
for linha in limiares:
    for i, item in enumerate(linha):
        if i==0:
            limiar1_lista.append(item[1])
        elif i==1:
            limiar2_lista.append(item[1])
        elif i==2:
            limiar3_lista.append(item[1])
            
#Criando um DataFrame com todas essas colunas           
similaridade_lentidao_e_CET = pd.DataFrame({'corredor':cet_corredores['corredor'],'match 1': match1_lista, 'match 2': match2_lista,'match 3': match3_lista,
              'similaridade 1': limiar1_lista,'similaridade 2': limiar2_lista,'similaridade 3': limiar3_lista} )       

In [ ]:
similaridade_lentidao_e_CET.to_csv(r"D:\Trabalho\faculdade\TCC\outputs\preprocessamento\georreferenciamento\similaridade_lentidao_e_CET.csv",index=False)

#### Considerando coluna cvc_nomelg

In [ ]:
fuzzy_merge,limiares = fuzzy_merge(corredores_lentidao_cet,ruas_cet,"corredor","cvc_nomelg")

In [ ]:
fuzzy_merge.head(30)